# Frontend scripts

In [34]:
%%writefile scripts/config-to-env.mjs
#!/usr/bin/env node
/**
 * Revised generator:
 *  - Introduces separation of LOCAL/STAGING/RAILWAY base keys (not auto-exposed).
 *  - Derives exactly one VITE_API_URL based on APP_ENV.
 *  - Still copies config.yaml into web/ for transparency.
 *  - Ensures old config.yaml is deleted before copying fresh one.
 *  - Adds APP_ENV to the .env file for frontend awareness.
 *
 * Selection rules:
 *   prod|production -> RAILWAY_VITE_API_BASE
 *   staging         -> STAGING_VITE_API_BASE || LOCAL_VITE_API_BASE
 *   default (dev)   -> LOCAL_VITE_API_BASE
 *
 * Only the final derived VITE_API_URL is written to web/.env (plus any other VITE_* keys that already exist).
 *
 * Safe: Non-VITE_* keys stay private (not shipped to client bundle).
 */

import fs from 'fs';
import path from 'path';
import { fileURLToPath } from 'url';
import * as yaml from 'yaml';

const __filename = fileURLToPath(import.meta.url);
const __dirname  = path.dirname(__filename);
const root       = path.resolve(__dirname, '..');
const webDir     = path.join(root, 'web');

function log(...args)  { console.log('[config-to-env]', ...args); }
function fail(msg)     { console.error('❌ [config-to-env]', msg); process.exit(1); }

const cliEnvArg = process.argv.slice(2).find(a => !a.startsWith('-'));
const targetEnv = (process.env.APP_ENV || cliEnvArg || 'dev').toLowerCase();

const cfgPath = path.join(root, 'config.yaml');
if (!fs.existsSync(cfgPath)) fail(`config.yaml not found at ${cfgPath}`);

let doc;
try {
  doc = yaml.parse(fs.readFileSync(cfgPath, 'utf8'));
} catch (e) {
  fail(`YAML parse error: ${e.message}`);
}

if (!doc.default) fail('Missing "default" section in config.yaml');

const merged = { ...doc.default, ...(doc[targetEnv] || {}) };

// ---- Derive effective base -------------------------------------------------
const localBase    = merged.LOCAL_VITE_API_BASE;
const stagingBase  = merged.STAGING_VITE_API_BASE || localBase;
const railwayBase  = merged.RAILWAY_VITE_API_BASE;

let effectiveBase;
if (['prod','production'].includes(targetEnv)) {
  effectiveBase = railwayBase;
} else if (targetEnv === 'staging') {
  effectiveBase = stagingBase;
} else {
  effectiveBase = localBase;
}

if (!effectiveBase) {
  fail(`No effective API base resolved (check LOCAL_VITE_API_BASE / RAILWAY_VITE_API_BASE keys).`);
}

// Normalize and append /api/v1 if needed
function normalizeApiBase(b) {
  let base = b.trim().replace(/\/+$/, '');
  if (!/\/api\/v1$/.test(base)) base += '/api/v1';
  return base;
}
const finalApiUrl = normalizeApiBase(effectiveBase);

// Collect any *existing* merged VITE_ keys (other than API URL we now control)
const viteEntries = Object.entries(merged)
  .filter(([k]) => k.startsWith('VITE_') && k !== 'VITE_API_URL')
  .map(([k, v]) => [k, v]);

// Inject our derived VITE_API_URL at the top for visibility
viteEntries.unshift(['VITE_API_URL', finalApiUrl]);

// Compose .env content
const lines = viteEntries.map(([k, v]) => `${k}=${v}`);

// Add APP_ENV to the .env file for frontend awareness
lines.push(`APP_ENV=${targetEnv}`);

const envOut = lines.join('\n') + '\n';

if (!fs.existsSync(webDir)) fail(`web directory not found at ${webDir}`);

// Delete old config.yaml from web/ if it exists
const webConfigPath = path.join(webDir, 'config.yaml');
if (fs.existsSync(webConfigPath)) {
  try {
    fs.unlinkSync(webConfigPath);
    log('Deleted old web/config.yaml');
  } catch (e) {
    log('Warning: Could not delete old web/config.yaml:', e.message);
  }
}

const envPath = path.join(webDir, '.env');
fs.writeFileSync(envPath, envOut, 'utf8');

// Copy fresh config.yaml for inspection
fs.copyFileSync(cfgPath, webConfigPath);

log(`Environment         : ${targetEnv}`);
log(`Resolved API base   : ${effectiveBase}`);
log(`VITE_API_URL (final): ${finalApiUrl}`);
log(`VITE keys written   : ${viteEntries.length}`);
log(`APP_ENV set to      : ${targetEnv}`);
log(`Config copied to    : ${webConfigPath}`); 



Overwriting scripts/config-to-env.mjs


In [35]:
%%writefile scripts/sync-env.js
// scripts/sync-env.js
import { copyFileSync } from 'fs';
import { resolve } from 'path';

console.log('🔄 Syncing environment file...');

const from = resolve(process.cwd(), 'web', 'env.template');
const to   = resolve(process.cwd(), 'web', '.env');

try {
  copyFileSync(from, to);
  console.log(`✅ Copied ${from} → ${to}`);
} catch (e) {
  console.error(`❌ Failed to copy env.template:`, e.message);
  process.exit(1);
} 

Overwriting scripts/sync-env.js


In [36]:
%%writefile scripts/verify-frontend-lock.js
#!/usr/bin/env node
/**
 * Sanity check that web/package-lock.json matches web/package.json ranges.
 * Warns when your lock is missing or stale (which breaks reproducibility with `npm ci`).
 */
import fs from 'fs';
import path from 'path';
import semver from 'semver';
import { fileURLToPath } from 'url';

const __filename = fileURLToPath(import.meta.url);
const __dirname = path.dirname(__filename);
const root = path.resolve(__dirname, '..');
const webDir = path.join(root, 'web');
const pkgPath = path.join(webDir, 'package.json');
const lockPath = path.join(webDir, 'package-lock.json');

function die(msg) {
  console.error(`❌ ${msg}`);
  process.exitCode = 1;
}

if (!fs.existsSync(pkgPath)) die('web/package.json not found');
if (!fs.existsSync(lockPath)) die('web/package-lock.json not found (run npm install in web/)');

const pkg = JSON.parse(fs.readFileSync(pkgPath, 'utf8'));
const lock = JSON.parse(fs.readFileSync(lockPath, 'utf8'));

const wanted = { ...pkg.dependencies, ...pkg.devDependencies };
const lockDeps = lock.packages || {};

let mismatches = 0;
for (const [name, range] of Object.entries(wanted)) {
  const lockInfo = lockDeps[`node_modules/${name}`];
  if (!lockInfo?.version) {
    console.warn(`⚠️  ${name} missing in lockfile`);
    mismatches++;
    continue;
  }
  if (!semver.satisfies(lockInfo.version, range, { includePrerelease: true })) {
    console.warn(`⚠️  ${name}@${lockInfo.version} does not satisfy ${range}`);
    mismatches++;
  }
}

if (mismatches === 0) {
  console.log('✅ Lockfile satisfies manifest ranges – suitable for npm ci reproducible installs.');
} else {
  console.log(`⚠️  ${mismatches} mismatch(es) found – run npm run frontend:rebuild-lock before CI.`);
} 

Overwriting scripts/verify-frontend-lock.js


In [37]:
%%writefile scripts/frontend-diagnose.js
#!/usr/bin/env node
/**
 * Quick dependency + audit snapshot for the frontend.
 * Prints versions of key packages and shows who depends on deprecated modules.
 */
import { execSync } from 'child_process';
import path from 'path';
import { fileURLToPath } from 'url';

const __filename = fileURLToPath(import.meta.url);
const __dirname = path.dirname(__filename);
const root = path.resolve(__dirname, '..');
const webDir = path.join(root, 'web');

function run(cmd) {
  console.log(`\n$ ${cmd}`);
  try {
    const out = execSync(cmd, { stdio: 'pipe', cwd: webDir, encoding: 'utf8', maxBuffer: 10_000_000 });
    console.log(out.trim());
  } catch (err) {
    console.error(err.stdout?.toString() || err.message);
  }
}

console.log('🔍 Frontend dependency diagnostics');
run('node -v');
run('npm -v');
run('npm ls --depth=2 inflight || true');
run('npm ls --depth=2 glob || true');
run('npm ls --depth=2 rimraf || true');
run('npm ls --depth=1 eslint || true');
run('npm audit --omit=dev || true');  // prod issues
run('npm audit || true');             // full tree
console.log('\nDone.'); 


Overwriting scripts/frontend-diagnose.js


In [38]:
%%writefile scripts/frontend-clean.js
#!/usr/bin/env node
/**
 * Clean the frontend install in web/ in a *cross-platform* way.
 * - Deletes node_modules
 * - Optionally deletes package-lock.json when --zap-lock passed
 * - Never follows symlinks outside repo
 *
 * Use via: npm run frontend:clean [-- --zap-lock]
 */

import fs from 'fs';
import path from 'path';
import { fileURLToPath } from 'url';

const __filename = fileURLToPath(import.meta.url);
const __dirname = path.dirname(__filename);
const root = path.resolve(__dirname, '..');
const webDir = path.join(root, 'web');

const args = process.argv.slice(2);
const zapLock = args.includes('--zap-lock');

function safeRemove(targetPath) {
  if (!fs.existsSync(targetPath)) return false;
  try {
    // fs.rmSync is cross-platform, supports recursive+force (Node 14+)
    fs.rmSync(targetPath, { recursive: true, force: true, maxRetries: 3, retryDelay: 100 });
    console.log(`Removed: ${path.relative(root, targetPath)}`);
    return true;
  } catch (err) {
    console.error(`Failed to remove ${targetPath}:`, err);
    process.exitCode = 1;
    return false;
  }
}

console.log('🧹 Frontend clean start');
safeRemove(path.join(webDir, 'node_modules'));
if (zapLock) {
  safeRemove(path.join(webDir, 'package-lock.json'));
}
console.log('🧹 Frontend clean complete'); 

Overwriting scripts/frontend-clean.js


# Web files

In [39]:
%%writefile web/postcss.config.cjs
module.exports = {
  plugins: {
    tailwindcss: {},
    autoprefixer: {}
  }
}; 

Overwriting web/postcss.config.cjs


In [40]:
%%writefile web/.gitignore
# Byte-compiled / optimized / DLL files
__pycache__/
*.py[cod]
*$py.class
.env.example
# C extensions
*.so

# Distribution / packaging 
.Python
build/
develop-eggs/
dist/
downloads/
eggs/
.eggs/
lib/
lib64/
parts/
sdist/
var/
wheels/
share/python-wheels/
*.egg-info/
.installed.cfg
*.egg
MANIFEST

# PyInstaller
#  Usually these files are written by a python script from a template
#  before PyInstaller builds the exe, so as to inject date/other infos into it.
*.manifest
*.spec

# Installer logs
pip-log.txt
pip-delete-this-directory.txt

# Unit test / coverage reports
htmlcov/
.tox/
.nox/
.coverage
.coverage.*
.cache
nosetests.xml
coverage.xml
*.cover
*.py,cover
.hypothesis/
.pytest_cache/
cover/

# Translations
*.mo
*.pot

# Django stuff:
*.log
local_settings.py
db.sqlite3
db.sqlite3-journal

# Flask stuff:
instance/
.webassets-cache

# Scrapy stuff:
.scrapy

# Sphinx documentation
docs/_build/

# PyBuilder
.pybuilder/
target/

# Jupyter Notebook
.ipynb_checkpoints

# IPython
profile_default/
ipython_config.py

# pyenv
#   For a library or package, you might want to ignore these files since the code is
#   intended to run in multiple environments; otherwise, check them in:
# .python-version

# pipenv
#   According to pypa/pipenv#598, it is recommended to include Pipfile.lock in version control.
#   However, in case of collaboration, if having platform-specific dependencies or dependencies
#   having no cross-platform support, pipenv may install dependencies that don't work, or not
#   install all needed dependencies.
#Pipfile.lock

# poetry
#   Similar to Pipfile.lock, it is generally recommended to include poetry.lock in version control.
#   This is especially recommended for binary packages to ensure reproducibility, and is more
#   commonly ignored for libraries.
#   https://python-poetry.org/docs/basic-usage/#commit-your-poetrylock-file-to-version-control
#poetry.lock

# pdm
#   Similar to Pipfile.lock, it is generally recommended to include pdm.lock in version control.
#pdm.lock
#   pdm stores project-wide configurations in .pdm.toml, but it is recommended to not include it
#   in version control.
#   https://pdm.fming.dev/#use-with-ide
.pdm.toml

# PEP 582; used by e.g. github.com/David-OConnor/pyflow and github.com/pdm-project/pdm
__pypackages__/

# Celery stuff
celerybeat-schedule
celerybeat.pid

# SageMath parsed files
*.sage.py

# Environments
.env
.venv
env/
venv/
ENV/
env.bak/
venv.bak/

# Spyder project settings
.spyderproject
.spyproject

# Rope project settings
.ropeproject

# mkdocs documentation
/site

# mypy
.mypy_cache/
.dmypy.json
dmypy.json

# Pyre type checker
.pyre/

# pytype static type analyzer
.pytype/

# Cython debug symbols
cython_debug/

# PyCharm
#  JetBrains specific template is maintained in a separate JetBrains.gitignore that can
#  be found at https://github.com/github/gitignore/blob/main/Global/JetBrains.gitignore
#  and can be added to the global gitignore or merged into this file.  For a more nuclear
#  option (not recommended) you can uncomment the following to ignore the entire idea folder.
#.idea/


Overwriting web/.gitignore


In [41]:
%%writefile Dockerfile.railway
# Railway Frontend Dockerfile
# =========================
# NOTE: This file must be saved as UTF-8 without BOM.
# If you see “unknown instruction: ��#”, strip the BOM with:
#    sed -i '1s/^\xEF\xBB\xBF//' Dockerfile.railway

# BUILD STAGE
FROM node:18-alpine AS build

# Set working directory
WORKDIR /app

# Copy package files first (for better layer caching)
COPY package*.json ./

# Install dependencies (no mounted caches under node_modules)
RUN npm ci --prefer-offline --no-audit --loglevel=error

# Copy source code
COPY . .

# Build with environment variables
ARG VITE_API_URL
ENV VITE_API_URL=$VITE_API_URL

# Debug: show environment vars during build
RUN echo "BUILD DEBUG: VITE_API_URL = $VITE_API_URL"
RUN echo "BUILD DEBUG: All ENV vars:" && env | grep -E "(VITE_|NODE_|RAILWAY_)" || true

# Build the application with verbose output
RUN echo "BUILD DEBUG: Starting build process..." && \
    npm run build 2>&1 | tee build.log

# Debug: check built files for API URL
RUN echo "BUILD DEBUG: Checking built files for API URL..." && \
    find dist -name "*.js" -exec grep -l "fastapi-production-1d13.up.railway.app" {} \; || \
    echo "BUILD DEBUG: API URL not found in built files"

# Debug: sample content from built JS files
RUN echo "BUILD DEBUG: Sample from built JS files:" && \
    find dist -name "*.js" | head -1 | xargs head -50 || \
    echo "BUILD DEBUG: No JS files found"

# Debug: search for VITE_API_URL in built files
RUN echo "BUILD DEBUG: Searching for VITE_API_URL in built files:" && \
    find dist -name "*.js" -exec grep -H "VITE_API_URL" {} \; || \
    echo "BUILD DEBUG: VITE_API_URL not found in built files"

# SERVE STAGE
FROM node:18-alpine

WORKDIR /app

# Install serve globally
RUN npm install -g serve

# Copy only the built assets from build stage
COPY --from=build /app/dist ./dist

# Expose port
EXPOSE $PORT

# Start the application
CMD ["sh", "-c", "serve -s dist -l $PORT --no-clipboard --no-port-switching"]


Overwriting Dockerfile.railway


In [42]:
%%writefile web/Dockerfile.railway
# Railway Frontend Dockerfile
# =========================
# This Dockerfile implements an optimized build process with proper caching:
# - Uses explicit cache paths to avoid conflicts with Railway's build system
# - Implements multi-stage build for better layer caching
# - Separates dependency installation from build for better caching
# - Works from web directory context (Railway builds from web/)

# Build stage
FROM node:18-alpine AS build

# Accept build argument for API URL
ARG VITE_API_URL
ENV VITE_API_URL=$VITE_API_URL

# Set working directory
WORKDIR /app

# Debug: Show initial environment
RUN echo "🔍 INITIAL DEBUG: Node version:" && node --version
RUN echo "🔍 INITIAL DEBUG: NPM version:" && npm --version
RUN echo "🔍 INITIAL DEBUG: Working directory:" && pwd
RUN echo "🔍 INITIAL DEBUG: Available files:" && ls -la

# Copy package files
COPY package*.json ./

# Debug: Verify package files are copied
RUN echo "🔍 PACKAGE DEBUG: Package files after copy:" && ls -la package*
RUN echo "🔍 PACKAGE DEBUG: Package.json content:" && head -20 package.json
RUN echo "🔍 PACKAGE DEBUG: Package-lock.json exists:" && test -f package-lock.json && echo "YES" || echo "NO"
RUN echo "🔍 PACKAGE DEBUG: Build script:" && cat package.json | grep -A 3 -B 3 '"build"'

# Install dependencies with better error handling
RUN echo "🔍 BUILD DEBUG: Installing dependencies..." && \
    npm ci --prefer-offline --no-audit --loglevel=error

# Debug: Verify node_modules
RUN echo "🔍 INSTALL DEBUG: Node modules created:" && ls -la node_modules | head -10
RUN echo "🔍 INSTALL DEBUG: Key packages installed:" && \
    ls node_modules/ | grep -E "(react|vite|rollup|typescript)" || echo "Key packages not found"

# Copy source code
COPY . .

# Debug: Show what files are available after source copy
RUN echo "🔍 SOURCE DEBUG: All files after source copy:" && ls -la
RUN echo "🔍 SOURCE DEBUG: Source directory structure:" && find . -type f -name "*.ts*" -o -name "*.js*" | head -20
RUN echo "🔍 SOURCE DEBUG: TypeScript config files:" && ls -la tsconfig*

# Debug environment variables
RUN echo "🔍 BUILD DEBUG: VITE_API_URL = ${VITE_API_URL}"
RUN echo "🔍 BUILD DEBUG: All ENV vars:" && env | grep -E "(VITE_|NODE_|RAILWAY_)" || true

# Pre-build TypeScript check with detailed error reporting
RUN echo "🔍 BUILD DEBUG: Running TypeScript check..." && \
    npm run type-check:verbose 2>&1 | tee typecheck.log || \
    (echo "⚠️  TypeScript check failed, but continuing with build..." && \
     echo "🔍 BUILD DEBUG: TypeScript errors:" && \
     cat typecheck.log && \
     echo "🔍 BUILD DEBUG: Attempting build anyway...")

# Build with fallback strategy
RUN echo "🔍 BUILD DEBUG: Starting build process..." && \
    (npm run build 2>&1 | tee build.log || \
     (echo "⚠️  Standard build failed, trying force build..." && \
      npm run build:force 2>&1 | tee build-force.log))

# Verify build output
RUN echo "🔍 BUILD DEBUG: Verifying build output..." && \
    ls -la dist/ && \
    echo "🔍 BUILD DEBUG: Build files:" && \
    find dist -type f -name "*.js" -o -name "*.css" -o -name "*.html" | head -10

# Check for API URL in built files
RUN echo "🔍 BUILD DEBUG: Checking built files for API URL..." && \
    find dist -name "*.js" -exec grep -l "${VITE_API_URL:-fastapi-production-1d13.up.railway.app}" {} \; || \
    echo "🔍 BUILD DEBUG: API URL not found in built files"

# Sample content from built JS files
RUN echo "🔍 BUILD DEBUG: Sample content from built JS files:" && \
    find dist -name "*.js" | head -1 | xargs head -50 || \
    echo "🔍 BUILD DEBUG: No JS files found"

# Search for VITE_API_URL in built files
RUN echo "🔍 BUILD DEBUG: Searching for VITE_API_URL in built files:" && \
    find dist -name "*.js" -exec grep -H "VITE_API_URL" {} \; || \
    echo "🔍 BUILD DEBUG: VITE_API_URL not found in built files"

# Final verification that dist exists
RUN echo "🔍 BUILD DEBUG: Final verification..." && \
    test -d dist && echo "✅ dist directory exists" || \
    (echo "❌ dist directory missing - build failed!" && exit 1)

# Production stage
FROM nginx:alpine

# Copy built assets from build stage
COPY --from=build /app/dist ./dist

# Copy nginx configuration
COPY nginx.conf /etc/nginx/nginx.conf

# Expose port
EXPOSE 80

# Health check
HEALTHCHECK --interval=30s --timeout=10s --start-period=5s --retries=3 \
  CMD wget --no-verbose --tries=1 --spider http://localhost:80/ || exit 1

# Start nginx
CMD ["nginx", "-g", "daemon off;"] 


Overwriting web/Dockerfile.railway


In [43]:
%%writefile web/env.template
ENV_NAME="react_fastapi_railway"
CUDA_TAG="12.8.0"
DOCKER_BUILDKIT="1"
HOST_JUPYTER_PORT="8890"
HOST_TENSORBOARD_PORT="6008"
HOST_EXPLAINER_PORT="8050"
HOST_STREAMLIT_PORT="8501"
HOST_MLFLOW_PORT="5000"
HOST_APP_PORT="5100"
HOST_BACKEND_DEV_PORT="5002"
MLFLOW_TRACKING_URI="http://mlflow:5000"
MLFLOW_VERSION="2.12.2"
PYTHON_VER="3.10"
JAX_PLATFORM_NAME="gpu"
XLA_PYTHON_CLIENT_PREALLOCATE="true"
XLA_PYTHON_CLIENT_ALLOCATOR="platform"
XLA_PYTHON_CLIENT_MEM_FRACTION="0.95"
XLA_FLAGS="--xla_force_host_platform_device_count=1"
JAX_DISABLE_JIT="false"
JAX_ENABLE_X64="false"
TF_FORCE_GPU_ALLOW_GROWTH="false"
JAX_PREALLOCATION_SIZE_LIMIT_BYTES="8589934592"
RAILWAY_TOKEN=""
RAILWAY_API_URL=""
RAILWAY_VITE_API_URL="https://fastapi-production-1d13.up.railway.app"
VITE_API_URL=http://127.0.0.1:8000/api/v1
REACT_APP_API_URL="https://react-frontend-production-2805.up.railway.app"
SECRET_KEY="change-me-in-prod"
USERNAME_KEY="alice"
USER_PASSWORD="supersecretvalue"
DATABASE_URL="sqlite+aiosqlite:///./app.db"
RAILWAY_ENVIRONMENT="production"
RAILWAY_ENVIRONMENT_ID="fa10dc06-75ec-4c11-93d4-a0fde17996d0"
RAILWAY_ENVIRONMENT_NAME="production"
RAILWAY_PRIVATE_DOMAIN="empowering-appreciation.railway.internal"
RAILWAY_PROJECT_ID="fc9da558-31d6-4b28-9eda-2bbe56cc7390"
RAILWAY_PROJECT_NAME="responsible-abundance"
RAILWAY_SERVICE_ID="87c129ab-ba49-471a-88bb-853ace60180d"
RAILWAY_SERVICE_NAME="empowering-appreciation"



Overwriting web/env.template


In [44]:
%%writefile web/eslint.config.js
import js from '@eslint/js'
import globals from 'globals'
import reactHooks from 'eslint-plugin-react-hooks'
import reactRefresh from 'eslint-plugin-react-refresh'
import tseslint from 'typescript-eslint'

export default tseslint.config(
  { ignores: ['dist'] },
  {
    extends: [js.configs.recommended, ...tseslint.configs.recommended],
    files: ['**/*.{ts,tsx}'],
    languageOptions: {
      ecmaVersion: 2020,
      globals: globals.browser,
    },
    plugins: {
      'react-hooks': reactHooks,
      'react-refresh': reactRefresh,
    },
    rules: {
      ...reactHooks.configs.recommended.rules,
      'react-refresh/only-export-components': [
        'warn',
        { allowConstantExport: true },
      ],
    },
  },
)


Overwriting web/eslint.config.js


In [45]:
%%writefile web/index.html
<!doctype html>
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <link rel="icon" type="image/svg+xml" href="/vite.svg" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Vite + React + TS</title>
  </head>
  <body>
    <div id="root"></div>
    <script type="module" src="/src/main.tsx"></script>
  </body>
</html>


Overwriting web/index.html


In [46]:
%%writefile web/nixpacks.toml
[phases.setup]
nixPkgs = ["nodejs-18_x", "npm-9_x"]

[phases.install]
cmds = [
  "npm ci --prefer-offline --no-audit --loglevel=error"
]

[phases.build]
cmds = [
  "npm run build"
]

[start]
cmd = "npx serve -s dist -l $PORT --no-clipboard --no-port-switching"

[variables]
NODE_ENV = "production"
NPM_CONFIG_PRODUCTION = "false" 

Overwriting web/nixpacks.toml


In [47]:
%%writefile web/package.json
{
  "name": "vite-react-typescript-starter",
  "private": true,
  "version": "0.0.0",
  "type": "module",
  "scripts": {
    "dev": "vite",
    "build": "vite build",
    "type-check": "tsc -p tsconfig.app.json --noEmit",
    "lint": "eslint .",
    "preview": "vite preview"
  },
  "dependencies": {
    "react": "^18.3.1",
    "react-dom": "^18.3.1",
    "axios": "^1.7.7",
    "react-router-dom": "^6.27.0",
    "recharts": "^2.12.0",
    "@headlessui/react": "^2.2.0",
    "@heroicons/react": "^2.1.5",
    "lucide-react": "^0.471.0",
    "react-hot-toast": "^2.5.1"
  },
  "devDependencies": {
    "@types/node": "^22.1.0",
    "@types/react": "^18.3.5",
    "@types/react-dom": "^18.3.0",
    "@vitejs/plugin-react": "^4.6.0",
    "eslint": "^9.9.0",
    "@eslint/js": "^9.9.0",
    "globals": "^15.9.0",
    "typescript-eslint": "^8.9.0",
    "eslint-plugin-react-hooks": "^5.1.0",
    "eslint-plugin-react-refresh": "^0.4.7",
    "typescript": "^5.6.3",
    "vite": "^5.4.1",
    "rollup": "^4.16.0",
    "tailwindcss": "^3.4.14",
    "postcss": "^8.4.40",
    "autoprefixer": "^10.4.20"
  },
  "overrides": {
    "esbuild": "^0.25.5",
    "glob": "^9.0.0",
    "rimraf": "^4.0.0"
  }
}


Overwriting web/package.json


In [48]:
%%writefile web/railway.json
{
  "$schema": "https://railway.app/railway.schema.json",
  "build": {
    "builder": "DOCKERFILE",
    "dockerfilePath": "Dockerfile.railway",
    "buildArgs": {
      "VITE_API_URL": "${{ VITE_API_URL }}"
    }
  },
  "deploy": {
    "restartPolicyType": "ON_FAILURE",
    "restartPolicyMaxRetries": 10
  }
}



Overwriting web/railway.json


In [49]:
%%writefile web/tsconfig.app.json
{
  "compilerOptions": {
    "composite": true,
    "module": "ESNext",
    "moduleResolution": "Node",
    "allowJs": true,
    "esModuleInterop": true,
    "resolveJsonModule": true,
    "lib": ["ESNext", "DOM"],
    "types": ["node", "vite/client"],
    "skipLibCheck": true,
    "noEmit": true
  },
  "include": ["vite.config.ts"]
} 


Overwriting web/tsconfig.app.json


In [50]:
%%writefile web/tsconfig.json
{
  "files": [],
  "references": [
    { "path": "./tsconfig.app.json" },
    { "path": "./tsconfig.node.json" }
  ]
}


Overwriting web/tsconfig.json


In [51]:
%%writefile web/tsconfig.node.json
{
  "compilerOptions": {
    "composite": true,
    "module": "ESNext",
    "moduleResolution": "Node",
    "allowJs": true,
    "esModuleInterop": true,
    "resolveJsonModule": true,
    "lib": ["ESNext"],
    "types": ["node"]
  },
  "include": ["vite.config.ts"]
} 

Overwriting web/tsconfig.node.json


In [52]:
%%writefile web/vite.config.ts
import { defineConfig, loadEnv } from 'vite'
import react from '@vitejs/plugin-react'
import path from 'path'

/**
 * Vite config:
 *  - Relies on generated web/.env containing *one* VITE_API_URL derived from base keys.
 *  - Keeps normalization safeguard.
 */
export default defineConfig(({ mode }) => {
  const envDir = __dirname
  const env = loadEnv(mode, envDir, '') // loads web/.env
  const raw = env.VITE_API_URL || ''
  const trimmed = raw.replace(/\/+$/, '')
  const API_URL = /\/api\/v1$/.test(trimmed) ? trimmed : `${trimmed}/api/v1`

  if (!API_URL) {
    if (mode === 'development') {
      console.warn('[vite.config] VITE_API_URL missing – using http://127.0.0.1:8000/api/v1')
    } else {
      throw new Error('[vite.config] VITE_API_URL is required for non-dev builds')
    }
  }

  console.log('🔍 Vite Config:')
  console.log('  Mode              :', mode)
  console.log('  Loaded from       :', path.join(envDir, '.env'))
  console.log('  VITE_API_URL (raw):', raw)
  console.log('  API_URL (final)   :', API_URL)

  return {
    envDir,
    plugins: [react()],
    define: {
      __BUILD_API_URL__: JSON.stringify(API_URL),
    },
    server: {
      host: '0.0.0.0',
      port: 5173,
      proxy: {
        '/api/v1': {
          target: 'http://127.0.0.1:8000',
            changeOrigin: true,
            secure: false,
            rewrite: p => p
        }
      }
    },
    build: {
      outDir: 'dist',
      assetsDir: 'assets',
      sourcemap: false,
      chunkSizeWarningLimit: 1000,
      rollupOptions: {
        output: {
          manualChunks: {
            vendor: ['react', 'react-dom']
          }
        }
      }
    },
    esbuild: {
      logOverride: { 'this-is-undefined-in-esm': 'silent' },
      target: 'es2020',
      keepNames: true
    }
  }
})


Overwriting web/vite.config.ts


In [53]:
%%writefile web/src/services/api.js
// web/src/services/api.js
import toast from 'react-hot-toast';     // add toast here for 401 handler

/**
 * API base resolution:
 *  - Single authoritative VITE_API_URL comes from generated web/.env
 *    (derived from LOCAL/STAGING/RAILWAY base keys in root config.yaml).
 *  - We still keep a build-time constant (__BUILD_API_URL__) as a fallback.
 *  - Only VITE_* keys are exposed to client code (per Vite rules).
 */
const API_BASE_URL = (() => {
  const envURL   = import.meta.env?.VITE_API_URL || ''
  const buildURL = (typeof __BUILD_API_URL__ !== 'undefined' && __BUILD_API_URL__) || ''
  const chosenRaw = envURL || buildURL

  let base = chosenRaw.replace(/\/+$/, '')
  if (base && !/\/api\/v1$/.test(base)) {
    base = base + '/api/v1'
  }

  if (!base) {
    console.warn('[ApiService] No VITE_API_URL provided – falling back to /api/v1 (DEV ONLY)')
    return '/api/v1'
  }

  console.log('[ApiService] Resolved API base:', {
    envURL, buildURL, final: base
  })

  return base
})();

// ─────────── Helper to join paths safely ─────────────────────────
const join = (base, path) => {
  const normalBase = base.replace(/\/+$/, '');     // trim trailing /
  const normalPath = path.replace(/^\/+/, '');     // trim leading /
  return `${normalBase}/${normalPath}`;            // single slash in-between
};

class ApiService {
  constructor() {
    this.baseURL = API_BASE_URL;            // now correct
    this.defaultHeaders = {};  // Remove default Content-Type to avoid CORS preflight for GETs
  }

  async request(endpoint, options = {}) {
    // Avoid repeating /api/v1 if caller passes it
    const cleanEndpoint = endpoint.replace(/^\/?api\/v1\//, '')
    const url = join(this.baseURL, cleanEndpoint)
    const token = localStorage.getItem('jwt')

    console.debug('[API Request]', {
      method: options.method || 'GET',
      endpoint,
      cleanEndpoint,
      url
    })

    const cfg = {
      method: options.method || 'GET',
      ...options,
      headers: {
        ...this.defaultHeaders,
        ...(token && { Authorization: `Bearer ${token}` }),
        ...options.headers
      }
    };

    try {
      const res = await fetch(url, cfg)

      const rateLimitRemaining = res.headers.get('X-RateLimit-Remaining')
      const rateLimitLimit = res.headers.get('X-RateLimit-Limit')
      const retryAfter = res.headers.get('Retry-After')

      if (rateLimitRemaining !== null) {
        const remaining = parseInt(rateLimitRemaining, 10)
        const limit = parseInt(rateLimitLimit, 10)
        if (remaining <= 3 && remaining > 0) {
          toast.warning(`Rate limit warning: ${remaining}/${limit} remaining`)
        }
        console.debug(`[API] Rate limit: ${remaining}/${limit}`)
      }

      if (res.status === 401) {
        localStorage.removeItem('jwt')
        toast.error('Session expired – please log in again.')
        window.location.replace('/login')
        return
      }

      if (res.status === 429) {
        const retrySeconds = retryAfter ? parseInt(retryAfter, 10) : 60
        toast.error(`Rate limit exceeded. Wait ${retrySeconds}s.`)
        throw new Error(`429 retry after ${retrySeconds}s`)
      }

      if (!res.ok) {
        const text = await res.text()
        console.error(`❌ [API] ${res.status} ${url} – ${text}`)
        throw new Error(`${res.status}: ${text}`)
      }

      return res.status !== 204 ? res.json() : null
    } catch (err) {
      console.error('❌ [API] Request failed:', err)
      throw err
    }
  }

  // Convenience wrappers
  getHealth()        { return this.request('/health') }
  getReady()         { return this.request('/ready/frontend') }
  getReadyFull()     { return this.request('/ready/full') }
  getHello()         { return this.request('/hello') }

  login(credentials) {
    const body = new URLSearchParams({
      username: credentials.username,
      password: credentials.password
    })
    return this.request('/token', {
      method: 'POST',
      headers: { 'Content-Type': 'application/x-www-form-urlencoded' },
      body
    })
  }

  predictIris(payload) {
    return this.request('/iris/predict', {
      method: 'POST',
      headers: { 'Content-Type': 'application/json' },
      body: JSON.stringify(payload)
    })
  }

  predictCancer(payload) {
    return this.request('/cancer/predict', {
      method: 'POST',
      headers: { 'Content-Type': 'application/json' },
      body: JSON.stringify(payload)
    })
  }

  trainIris(modelType = 'rf') {
    return this.request('/iris/train', {
      method: 'POST',
      headers: { 'Content-Type': 'application/json' },
      body: JSON.stringify({ model_type: modelType })
    })
  }

  trainCancer(modelType = 'bayes') {
    return this.request('/cancer/train', {
      method: 'POST',
      headers: { 'Content-Type': 'application/json' },
      body: JSON.stringify({ model_type: modelType })
    })
  }

  test401() { return this.request('/test/401') }
}

export const apiService = new ApiService();
export default apiService;


Overwriting web/src/services/api.js


In [54]:
%%writefile web/src/components/Login.jsx
import React, { useState } from 'react';
import { Brain, Eye, EyeOff } from 'lucide-react';
import toast from 'react-hot-toast';

const Login = ({ onLogin, backendReady }) => {
  const [credentials, setCredentials] = useState({
    username: '',
    password: ''
  });
  const [showPassword, setShowPassword] = useState(false);
  const [isLoading, setIsLoading] = useState(false);

  const handleChange = (e) => {
    setCredentials({
      ...credentials,
      [e.target.name]: e.target.value
    });
  };

  const handleSubmit = async (e) => {
    e.preventDefault();
    setIsLoading(true);

    try {
      const result = await onLogin(credentials);
      if (result.success) {
        toast.success('Login successful!');
      } else {
        toast.error(result.error || 'Login failed');
      }
    } catch (error) {
      toast.error('An unexpected error occurred');
    } finally {
      setIsLoading(false);
    }
  };

  return (
    <div className="min-h-screen flex items-center justify-center bg-gray-50 py-12 px-4 sm:px-6 lg:px-8">
      <div className="max-w-md w-full space-y-8">
        <div>
          <div className="flex justify-center">
            <Brain className="h-12 w-12 text-primary-600" />
          </div>
          <h2 className="mt-6 text-center text-3xl font-extrabold text-gray-900">
            Sign in to ML Dashboard
          </h2>
          <p className="mt-2 text-center text-sm text-gray-600">
            Access your machine learning models and predictions
          </p>
        </div>

        <form className="mt-8 space-y-6" onSubmit={handleSubmit}>
          <div className="rounded-md shadow-sm -space-y-px">
            <div>
              <label htmlFor="username" className="sr-only">
                Username
              </label>
              <input
                id="username"
                name="username"
                type="text"
                required
                className="appearance-none rounded-none relative block w-full px-3 py-2 border border-gray-300 placeholder-gray-500 text-gray-900 rounded-t-md focus:outline-none focus:ring-primary-500 focus:border-primary-500 focus:z-10 sm:text-sm"
                placeholder="Username"
                value={credentials.username}
                onChange={handleChange}
              />
            </div>
            <div className="relative">
              <label htmlFor="password" className="sr-only">
                Password
              </label>
              <input
                id="password"
                name="password"
                type={showPassword ? "text" : "password"}
                required
                className="appearance-none rounded-none relative block w-full px-3 py-2 border border-gray-300 placeholder-gray-500 text-gray-900 rounded-b-md focus:outline-none focus:ring-primary-500 focus:border-primary-500 focus:z-10 sm:text-sm pr-10"
                placeholder="Password"
                value={credentials.password}
                onChange={handleChange}
              />
              <button
                type="button"
                className="absolute inset-y-0 right-0 pr-3 flex items-center"
                onClick={() => setShowPassword(!showPassword)}
              >
                {showPassword ? (
                  <EyeOff className="h-5 w-5 text-gray-400" />
                ) : (
                  <Eye className="h-5 w-5 text-gray-400" />
                )}
              </button>
            </div>
          </div>

          <div>
            <button
              type="submit"
              disabled={isLoading || !backendReady}
              className="group relative w-full flex justify-center py-2 px-4 border border-transparent text-sm font-medium rounded-md text-white bg-primary-600 hover:bg-primary-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-primary-500 disabled:opacity-50 disabled:cursor-not-allowed"
            >
              {!backendReady
                ? 'Backend loading – please wait…'
                : isLoading
                ? 'Signing in…'
                : 'Sign in'}
            </button>
            {!backendReady && (
              <p className="mt-2 text-xs text-yellow-600 text-center">
                Models are still warming up. You'll be able to sign in as soon as the light
                turns green.
              </p>
            )}
          </div>

          <div className="text-center">
            <p className="text-sm text-gray-600">
              Demo credentials: <strong>alice</strong> / <strong>supersecretvalue</strong>
            </p>
          </div>
        </form>
      </div>
    </div>
  );
};

export default Login; 




Overwriting web/src/components/Login.jsx


In [55]:
%%writefile web/src/components/Layout.jsx
import React from 'react';
import { LogOut, Activity, Brain, BarChart3 } from 'lucide-react';

const Layout = ({ children, user, onLogout }) => {
  return (
    <div className="min-h-screen bg-gray-50">
      {/* Header */}
      <header className="bg-white shadow-sm border-b border-gray-200">
        <div className="max-w-7xl mx-auto px-4 sm:px-6 lg:px-8">
          <div className="flex justify-between items-center h-16">
            {/* Logo/Title */}
            <div className="flex items-center">
              <Brain className="h-8 w-8 text-primary-600 mr-3" />
              <h1 className="text-xl font-semibold text-gray-900">
                ML Dashboard
              </h1>
            </div>

            {/* Navigation */}
            <nav className="hidden md:flex space-x-8">
              <a
                href="#"
                className="text-gray-500 hover:text-gray-700 px-3 py-2 rounded-md text-sm font-medium flex items-center"
              >
                <BarChart3 className="h-4 w-4 mr-2" />
                Models
              </a>
              <a
                href="#"
                className="text-gray-500 hover:text-gray-700 px-3 py-2 rounded-md text-sm font-medium flex items-center"
              >
                <Activity className="h-4 w-4 mr-2" />
                Predictions
              </a>
            </nav>

            {/* User menu */}
            <div className="flex items-center space-x-4">
              <div className="flex items-center space-x-2">
                <div className="h-8 w-8 bg-primary-600 rounded-full flex items-center justify-center">
                  <span className="text-white text-sm font-medium">
                    {user?.username?.charAt(0).toUpperCase() || 'U'}
                  </span>
                </div>
                <span className="text-sm text-gray-700">
                  {user?.username || 'User'}
                </span>
              </div>
              <button
                onClick={onLogout}
                className="text-gray-500 hover:text-gray-700 p-2 rounded-md"
                title="Logout"
              >
                <LogOut className="h-5 w-5" />
              </button>
            </div>
          </div>
        </div>
      </header>

      {/* Main content */}
      <main className="max-w-7xl mx-auto px-4 sm:px-6 lg:px-8 py-8">
        {children}
      </main>
    </div>
  );
};

export default Layout; 

Overwriting web/src/components/Layout.jsx


In [56]:
%%writefile web/src/components/ModelTraining.jsx
import React from 'react';
import { Zap, Clock, CheckCircle } from 'lucide-react';
import toast from 'react-hot-toast';

const ModelTraining = ({ dataset, onTrain, isTraining, setIsTraining }) => {
  const getDatasetInfo = () => {
    switch (dataset) {
      case 'iris':
        return {
          name: 'Iris Classification',
          description: 'Train a new model on the Iris dataset',
          estimatedTime: '30 seconds',
          features: 4,
          samples: 150
        };
      case 'cancer':
        return {
          name: 'Breast Cancer Diagnosis',
          description: 'Train a new Bayesian model on the breast cancer dataset',
          estimatedTime: '2-3 minutes',
          features: 30,
          samples: 569
        };
      default:
        return {
          name: 'Unknown Dataset',
          description: 'Train a new model',
          estimatedTime: 'Unknown',
          features: 0,
          samples: 0
        };
    }
  };

  const pollReady = async (attempt = 0) => {
    try {
      // For now, we'll just wait a bit and then stop training
      // The parent component is already polling model status
      await new Promise(resolve => setTimeout(resolve, 2000));
      setIsTraining(false);
    } catch { /* ignore */ }
  };

  const handleTrain = async () => {
    setIsTraining(true);
    try {
      // delegate to the parent so it can pass the right model_type
      await onTrain();
      toast.success('Training job submitted – refresh will turn green when done');
      pollReady();
    } catch (e) {
      toast.error(`Training failed: ${e.message}`);
      setIsTraining(false);
    }
  };

  const datasetInfo = getDatasetInfo();

  return (
    <div className="card">
      <div className="card-header">
        <h3 className="text-lg font-medium text-gray-900">Model Training</h3>
        <p className="text-sm text-gray-600">
          Train a new model or retrain existing models with updated parameters
        </p>
      </div>

      <div className="space-y-4">
        {/* Dataset Info */}
        <div className="bg-gray-50 rounded-lg p-4">
          <h4 className="font-medium text-gray-900 mb-2">{datasetInfo.name}</h4>
          <p className="text-sm text-gray-600 mb-3">{datasetInfo.description}</p>

          <div className="grid grid-cols-2 md:grid-cols-3 gap-4 text-sm">
            <div>
              <span className="text-gray-500">Features:</span>
              <span className="ml-2 font-medium">{datasetInfo.features}</span>
            </div>
            <div>
              <span className="text-gray-500">Samples:</span>
              <span className="ml-2 font-medium">{datasetInfo.samples}</span>
            </div>
            <div>
              <span className="text-gray-500">Est. Time:</span>
              <span className="ml-2 font-medium">{datasetInfo.estimatedTime}</span>
            </div>
          </div>
        </div>

        {/* Training Status */}
        {isTraining && (
          <div className="bg-blue-50 border border-blue-200 rounded-lg p-4">
            <div className="flex items-center">
              <div className="spinner h-5 w-5 mr-3"></div>
              <div className="flex-1">
                <h4 className="font-medium text-blue-900">Training in Progress</h4>
                <p className="text-sm text-blue-700">
                  Please wait while the model is being trained. This may take a few minutes.
                </p>
              </div>
            </div>

            <div className="mt-3">
              <div className="flex items-center text-sm text-blue-600">
                <Clock className="h-4 w-4 mr-2" />
                <span>Estimated time remaining: {datasetInfo.estimatedTime}</span>
              </div>
            </div>
          </div>
        )}

        {/* Training Options */}
        <div className="space-y-3">
          <h4 className="font-medium text-gray-900">Training Options</h4>

          {dataset === 'iris' && (
            <div className="space-y-2">
              <div className="flex items-center justify-between p-3 bg-gray-50 rounded-lg">
                <div>
                  <h5 className="font-medium text-gray-900">Random Forest</h5>
                  <p className="text-sm text-gray-600">
                    Fast ensemble method with good performance
                  </p>
                </div>
                <div className="text-green-600">
                  <CheckCircle className="h-5 w-5" />
                </div>
              </div>

              <div className="flex items-center justify-between p-3 bg-gray-50 rounded-lg">
                <div>
                  <h5 className="font-medium text-gray-900">Logistic Regression</h5>
                  <p className="text-sm text-gray-600">
                    Simple linear model with interpretable results
                  </p>
                </div>
                <div className="text-green-600">
                  <CheckCircle className="h-5 w-5" />
                </div>
              </div>
            </div>
          )}

          {dataset === 'cancer' && (
            <div className="space-y-2">
              <div className="flex items-center justify-between p-3 bg-gray-50 rounded-lg">
                <div>
                  <h5 className="font-medium text-gray-900">Bayesian Model</h5>
                  <p className="text-sm text-gray-600">
                    Hierarchical Bayesian model with uncertainty quantification
                  </p>
                </div>
                <div className="text-green-600">
                  <CheckCircle className="h-5 w-5" />
                </div>
              </div>
            </div>
          )}
        </div>

        {/* Training Button */}
        <div className="flex justify-center pt-4">
          <button
            onClick={handleTrain}
            disabled={isTraining}
            className="btn-primary btn-lg"
          >
            {isTraining ? (
              <>
                <div className="spinner h-5 w-5 mr-2"></div>
                Training Model...
              </>
            ) : (
              <>
                <Zap className="h-5 w-5 mr-2" />
                Train Model
              </>
            )}
          </button>
        </div>

        {/* Training Notes */}
        <div className="bg-yellow-50 border border-yellow-200 rounded-lg p-3">
          <h4 className="font-medium text-yellow-800 mb-2">Training Notes</h4>
          <ul className="text-sm text-yellow-700 space-y-1">
            <li>• Models are automatically saved after training</li>
            <li>• Previous models will be backed up before retraining</li>
            <li>• Training progress is logged in the console</li>
            {dataset === 'cancer' && (
              <li>• Bayesian models provide uncertainty estimates</li>
            )}
          </ul>
        </div>
      </div>
    </div>
  );
};

export default ModelTraining; 




Overwriting web/src/components/ModelTraining.jsx


In [57]:
%%writefile web/src/components/ResultsDisplay.jsx
import React from 'react';
import { CheckCircle, XCircle, AlertTriangle, BarChart3 } from 'lucide-react';

const ResultsDisplay = ({ predictions, dataset, isLoading }) => {
  if (isLoading) {
    return (
      <div className="flex items-center justify-center py-12">
        <div className="text-center">
          <div className="spinner h-8 w-8 mx-auto mb-4"></div>
          <p className="text-gray-600">Making prediction...</p>
        </div>
      </div>
    );
  }

  if (!predictions) {
    return (
      <div className="flex items-center justify-center py-12">
        <div className="text-center text-gray-500">
          <BarChart3 className="h-12 w-12 mx-auto mb-4 text-gray-300" />
          <p>Run a prediction to see results</p>
        </div>
      </div>
    );
  }

  const renderIrisResults = () => {
    const prediction = predictions.predictions[0];
    const probabilities = predictions.probabilities[0];
    const classes = ['setosa', 'versicolor', 'virginica'];

    const getSpeciesColor = (species) => {
      switch (species.toLowerCase()) {
        case 'setosa': return 'text-green-600 bg-green-100';
        case 'versicolor': return 'text-blue-600 bg-blue-100';
        case 'virginica': return 'text-purple-600 bg-purple-100';
        default: return 'text-gray-600 bg-gray-100';
      }
    };

    return (
      <div className="space-y-4">
        {/* Main Prediction */}
        <div className="text-center">
          <div className={`inline-flex items-center px-4 py-2 rounded-full font-medium ${getSpeciesColor(prediction)}`}>
            <CheckCircle className="h-5 w-5 mr-2" />
            {prediction.charAt(0).toUpperCase() + prediction.slice(1)}
          </div>
          <p className="text-sm text-gray-600 mt-2">Predicted Iris Species</p>
        </div>

        {/* Confidence Scores */}
        <div className="space-y-2">
          <h4 className="text-sm font-medium text-gray-700">Confidence Scores</h4>
          {classes.map((cls, index) => (
            <div key={cls} className="flex items-center justify-between">
              <span className="text-sm text-gray-600 capitalize">{cls}</span>
              <div className="flex items-center space-x-2">
                <div className="w-24 bg-gray-200 rounded-full h-2">
                  <div
                    className={`h-2 rounded-full ${
                      cls === prediction.toLowerCase() ? 'bg-primary-600' : 'bg-gray-400'
                    }`}
                    style={{ width: `${(probabilities[index] * 100)}%` }}
                  ></div>
                </div>
                <span className="text-sm font-medium text-gray-700 w-12">
                  {(probabilities[index] * 100).toFixed(1)}%
                </span>
              </div>
            </div>
          ))}
        </div>
      </div>
    );
  };

  const renderCancerResults = () => {
    const prediction  = predictions.predictions[0]?.toLowerCase();   // 'malignant' | 'benign'
    const probability = predictions.probabilities[0];
    const isMalignant = prediction.startsWith('m');  // works for full word or 'M'

    return (
      <div className="space-y-4">
        {/* Main Prediction */}
        <div className="text-center">
          <div className={`inline-flex items-center px-4 py-2 rounded-full font-medium ${
            isMalignant 
              ? 'text-red-600 bg-red-100' 
              : 'text-green-600 bg-green-100'
          }`}>
            {isMalignant ? (
              <XCircle className="h-5 w-5 mr-2" />
            ) : (
              <CheckCircle className="h-5 w-5 mr-2" />
            )}
            {isMalignant ? 'Malignant' : 'Benign'}
          </div>
          <p className="text-sm text-gray-600 mt-2">Predicted Diagnosis</p>
        </div>

        {/* Probability */}
        <div className="space-y-2">
          <h4 className="text-sm font-medium text-gray-700">Confidence</h4>
          <div className="flex items-center justify-between">
            <span className="text-sm text-gray-600">
              {isMalignant ? 'Malignancy' : 'Benign'} Probability
            </span>
            <div className="flex items-center space-x-2">
              <div className="w-24 bg-gray-200 rounded-full h-2">
                <div
                  className={`h-2 rounded-full ${
                    isMalignant ? 'bg-red-500' : 'bg-green-500'
                  }`}
                  style={{ width: `${(probability * 100)}%` }}
                ></div>
              </div>
              <span className="text-sm font-medium text-gray-700 w-12">
                {(probability * 100).toFixed(1)}%
              </span>
            </div>
          </div>
        </div>

        {/* Uncertainty (if available) */}
        {predictions.uncertainties && predictions.uncertainties[0] && (
          <div className="bg-yellow-50 border border-yellow-200 rounded-lg p-3">
            <div className="flex items-center">
              <AlertTriangle className="h-4 w-4 text-yellow-600 mr-2" />
              <span className="text-sm font-medium text-yellow-800">
                Uncertainty Information
              </span>
            </div>
            <p className="text-sm text-yellow-700 mt-1">
              Model uncertainty: {predictions.uncertainties[0].toFixed(3)}
            </p>
          </div>
        )}
      </div>
    );
  };

  return (
    <div className="space-y-4">
      {dataset === 'iris' ? renderIrisResults() : renderCancerResults()}

      {/* Input Echo */}
      <div className="border-t pt-4">
        <h4 className="text-sm font-medium text-gray-700 mb-2">Input Values</h4>
        <div className="bg-gray-50 rounded-lg p-3">
          <pre className="text-xs text-gray-600 overflow-x-auto">
            {JSON.stringify(predictions.input_received[0], null, 2)}
          </pre>
        </div>
      </div>
    </div>
  );
};

export default ResultsDisplay; 


Overwriting web/src/components/ResultsDisplay.jsx


In [58]:
%%writefile web/src/components/CancerForm.jsx
import React, { useState } from 'react';
import { Play, RotateCcw, ChevronDown, ChevronUp } from 'lucide-react';

const CancerForm = ({ onPredict, isLoading, onModelTypeChange }) => {
  const [showAdvanced, setShowAdvanced] = useState(false);
  const [formData, setFormData] = useState({
    model_type: 'bayes',
    samples: [{
      // Mean features
      mean_radius: 14.13,
      mean_texture: 19.26,
      mean_perimeter: 91.97,
      mean_area: 654.89,
      mean_smoothness: 0.096,
      mean_compactness: 0.104,
      mean_concavity: 0.089,
      mean_concave_points: 0.048,
      mean_symmetry: 0.181,
      mean_fractal_dimension: 0.063,

      // SE features
      se_radius: 0.406,
      se_texture: 1.216,
      se_perimeter: 2.866,
      se_area: 40.34,
      se_smoothness: 0.007,
      se_compactness: 0.025,
      se_concavity: 0.032,
      se_concave_points: 0.012,
      se_symmetry: 0.020,
      se_fractal_dimension: 0.004,

      // Worst features
      worst_radius: 16.27,
      worst_texture: 25.68,
      worst_perimeter: 107.26,
      worst_area: 880.58,
      worst_smoothness: 0.132,
      worst_compactness: 0.254,
      worst_concavity: 0.273,
      worst_concave_points: 0.114,
      worst_symmetry: 0.290,
      worst_fractal_dimension: 0.084
    }]
  });

  const handleInputChange = (field, value) => {
    setFormData(prev => ({
      ...prev,
      samples: [{
        ...prev.samples[0],
        [field]: parseFloat(value) || 0
      }]
    }));
  };

  const handleModelTypeChange = (value) => {
    setFormData(prev => ({
      ...prev,
      model_type: value
    }));
    if (onModelTypeChange) {
      onModelTypeChange(value);
    }
  };

  const handleSubmit = (e) => {
    e.preventDefault();
    onPredict(formData);
  };

  const handleReset = () => {
    setFormData({
      model_type: 'bayes',
      samples: [{
        mean_radius: 14.13,
        mean_texture: 19.26,
        mean_perimeter: 91.97,
        mean_area: 654.89,
        mean_smoothness: 0.096,
        mean_compactness: 0.104,
        mean_concavity: 0.089,
        mean_concave_points: 0.048,
        mean_symmetry: 0.181,
        mean_fractal_dimension: 0.063,
        se_radius: 0.406,
        se_texture: 1.216,
        se_perimeter: 2.866,
        se_area: 40.34,
        se_smoothness: 0.007,
        se_compactness: 0.025,
        se_concavity: 0.032,
        se_concave_points: 0.012,
        se_symmetry: 0.020,
        se_fractal_dimension: 0.004,
        worst_radius: 16.27,
        worst_texture: 25.68,
        worst_perimeter: 107.26,
        worst_area: 880.58,
        worst_smoothness: 0.132,
        worst_compactness: 0.254,
        worst_concavity: 0.273,
        worst_concave_points: 0.114,
        worst_symmetry: 0.290,
        worst_fractal_dimension: 0.084
      }]
    });
  };

  const sample = formData.samples[0];

  const meanFeatures = [
    { key: 'mean_radius', label: 'Mean Radius', step: 0.001, max: 30 },
    { key: 'mean_texture', label: 'Mean Texture', step: 0.001, max: 50 },
    { key: 'mean_perimeter', label: 'Mean Perimeter', step: 0.001, max: 200 },
    { key: 'mean_area', label: 'Mean Area', step: 0.001, max: 2500 },
    { key: 'mean_smoothness', label: 'Mean Smoothness', step: 0.001, max: 0.2 },
    { key: 'mean_compactness', label: 'Mean Compactness', step: 0.001, max: 0.5 },
    { key: 'mean_concavity', label: 'Mean Concavity', step: 0.001, max: 0.5 },
    { key: 'mean_concave_points', label: 'Mean Concave Points', step: 0.001, max: 0.2 },
    { key: 'mean_symmetry', label: 'Mean Symmetry', step: 0.001, max: 0.4 },
    { key: 'mean_fractal_dimension', label: 'Mean Fractal Dimension', step: 0.001, max: 0.1 }
  ];

  const seFeatures = [
    { key: 'se_radius', label: 'SE Radius', step: 0.001, max: 3 },
    { key: 'se_texture', label: 'SE Texture', step: 0.001, max: 5 },
    { key: 'se_perimeter', label: 'SE Perimeter', step: 0.001, max: 20 },
    { key: 'se_area', label: 'SE Area', step: 0.1, max: 500 },
    { key: 'se_smoothness', label: 'SE Smoothness', step: 0.0001, max: 0.05 },
    { key: 'se_compactness', label: 'SE Compactness', step: 0.001, max: 0.1 },
    { key: 'se_concavity', label: 'SE Concavity', step: 0.001, max: 0.2 },
    { key: 'se_concave_points', label: 'SE Concave Points', step: 0.001, max: 0.05 },
    { key: 'se_symmetry', label: 'SE Symmetry', step: 0.001, max: 0.1 },
    { key: 'se_fractal_dimension', label: 'SE Fractal Dimension', step: 0.0001, max: 0.02 }
  ];

  const worstFeatures = [
    { key: 'worst_radius', label: 'Worst Radius', step: 0.1, max: 40 },
    { key: 'worst_texture', label: 'Worst Texture', step: 0.1, max: 60 },
    { key: 'worst_perimeter', label: 'Worst Perimeter', step: 0.1, max: 300 },
    { key: 'worst_area', label: 'Worst Area', step: 1, max: 4000 },
    { key: 'worst_smoothness', label: 'Worst Smoothness', step: 0.001, max: 0.3 },
    { key: 'worst_compactness', label: 'Worst Compactness', step: 0.001, max: 1.0 },
    { key: 'worst_concavity', label: 'Worst Concavity', step: 0.001, max: 1.0 },
    { key: 'worst_concave_points', label: 'Worst Concave Points', step: 0.001, max: 0.3 },
    { key: 'worst_symmetry', label: 'Worst Symmetry', step: 0.001, max: 0.7 },
    { key: 'worst_fractal_dimension', label: 'Worst Fractal Dimension', step: 0.001, max: 0.2 }
  ];

  const renderFeatureGroup = (features, title) => (
    <div className="space-y-3">
      <h4 className="text-sm font-medium text-gray-700">{title}</h4>
      <div className="grid grid-cols-1 sm:grid-cols-2 lg:grid-cols-3 gap-3">
        {features.map((feature) => (
          <div key={feature.key} className="form-group">
            <label className="form-label text-xs">{feature.label}</label>
            <input
              type="number"
              step={feature.step}
              min="0"
              max={feature.max}
              value={sample[feature.key]}
              onChange={(e) => handleInputChange(feature.key, e.target.value)}
              className="form-input text-sm"
            />
          </div>
        ))}
      </div>
    </div>
  );

  return (
    <form onSubmit={handleSubmit} className="space-y-4">
      {/* Model Type Selection */}
      <div className="form-group">
        <label className="form-label">Model Type</label>
        <select
          value={formData.model_type}
          onChange={(e) => handleModelTypeChange(e.target.value)}
          className="form-input"
        >
          <option value="bayes">Bayesian Model</option>
          <option value="stub">Logreg (Fast fallback)</option>
        </select>
      </div>

      {/* Primary Features (Mean) */}
      {renderFeatureGroup(meanFeatures.slice(0, 4), "Primary Features")}

      {/* Advanced Features Toggle */}
      <div className="border-t pt-4">
        <button
          type="button"
          onClick={() => setShowAdvanced(!showAdvanced)}
          className="flex items-center text-sm text-primary-600 hover:text-primary-700"
        >
          {showAdvanced ? (
            <>
              <ChevronUp className="h-4 w-4 mr-1" />
              Hide Advanced Features
            </>
          ) : (
            <>
              <ChevronDown className="h-4 w-4 mr-1" />
              Show All Features (30 total)
            </>
          )}
        </button>
      </div>

      {/* Advanced Features */}
      {showAdvanced && (
        <div className="space-y-6 bg-gray-50 p-4 rounded-lg">
          {renderFeatureGroup(meanFeatures.slice(4), "Additional Mean Features")}
          {renderFeatureGroup(seFeatures, "Standard Error Features")}
          {renderFeatureGroup(worstFeatures, "Worst Features")}
        </div>
      )}

      {/* Action Buttons */}
      <div className="flex space-x-3 pt-4">
        <button
          type="submit"
          disabled={isLoading}
          className="btn-primary flex-1"
        >
          {isLoading ? (
            <>
              <div className="spinner h-4 w-4 mr-2"></div>
              Predicting...
            </>
          ) : (
            <>
              <Play className="h-4 w-4 mr-2" />
              Predict Diagnosis
            </>
          )}
        </button>

        <button
          type="button"
          onClick={handleReset}
          className="btn-outline"
          disabled={isLoading}
        >
          <RotateCcw className="h-4 w-4 mr-2" />
          Reset
        </button>
      </div>

      {/* Sample Data Info */}
      <div className="bg-gray-50 rounded-lg p-3 mt-4">
        <h4 className="text-sm font-medium text-gray-700 mb-2">About the Features:</h4>
        <div className="text-xs text-gray-600 space-y-1">
          <div><strong>Mean:</strong> Average values of cell nucleus features</div>
          <div><strong>SE:</strong> Standard error of the features</div>
          <div><strong>Worst:</strong> Worst (largest) values of the features</div>
        </div>
      </div>
    </form>
  );
};

export default CancerForm; 


Overwriting web/src/components/CancerForm.jsx


In [59]:
%%writefile web/src/components/IrisForm.jsx
import React, { useState } from 'react';
import { Play, RotateCcw } from 'lucide-react';

const IrisForm = ({ onPredict, isLoading, onModelTypeChange }) => {
  const [formData, setFormData] = useState({
    model_type: 'rf',
    samples: [{
      sepal_length: 5.1,
      sepal_width: 3.5,
      petal_length: 1.4,
      petal_width: 0.2
    }]
  });

  const handleInputChange = (field, value) => {
    setFormData(prev => ({
      ...prev,
      samples: [{
        ...prev.samples[0],
        [field]: parseFloat(value) || 0
      }]
    }));
  };

  const handleModelTypeChange = (value) => {
    setFormData(prev => ({
      ...prev,
      model_type: value
    }));
    if (onModelTypeChange) {
      onModelTypeChange(value);
    }
  };

  const handleSubmit = (e) => {
    e.preventDefault();
    onPredict(formData);
  };

  const handleReset = () => {
    setFormData({
      model_type: 'rf',
      samples: [{
        sepal_length: 5.1,
        sepal_width: 3.5,
        petal_length: 1.4,
        petal_width: 0.2
      }]
    });
  };

  const sample = formData.samples[0];

  return (
    <form onSubmit={handleSubmit} className="space-y-4">
      {/* Model Type Selection */}
      <div className="form-group">
        <label className="form-label">Model Type</label>
        <select
          value={formData.model_type}
          onChange={(e) => handleModelTypeChange(e.target.value)}
          className="form-input"
        >
          <option value="rf">Random Forest</option>
          <option value="logreg">Logistic Regression</option>
        </select>
      </div>

      {/* Feature Inputs */}
      <div className="grid grid-cols-1 sm:grid-cols-2 gap-4">
        <div className="form-group">
          <label className="form-label">Sepal Length (cm)</label>
          <input
            type="number"
            step="0.1"
            min="0"
            max="10"
            value={sample.sepal_length}
            onChange={(e) => handleInputChange('sepal_length', e.target.value)}
            className="form-input"
            placeholder="e.g., 5.1"
          />
        </div>

        <div className="form-group">
          <label className="form-label">Sepal Width (cm)</label>
          <input
            type="number"
            step="0.1"
            min="0"
            max="10"
            value={sample.sepal_width}
            onChange={(e) => handleInputChange('sepal_width', e.target.value)}
            className="form-input"
            placeholder="e.g., 3.5"
          />
        </div>

        <div className="form-group">
          <label className="form-label">Petal Length (cm)</label>
          <input
            type="number"
            step="0.1"
            min="0"
            max="10"
            value={sample.petal_length}
            onChange={(e) => handleInputChange('petal_length', e.target.value)}
            className="form-input"
            placeholder="e.g., 1.4"
          />
        </div>

        <div className="form-group">
          <label className="form-label">Petal Width (cm)</label>
          <input
            type="number"
            step="0.1"
            min="0"
            max="10"
            value={sample.petal_width}
            onChange={(e) => handleInputChange('petal_width', e.target.value)}
            className="form-input"
            placeholder="e.g., 0.2"
          />
        </div>
      </div>

      {/* Action Buttons */}
      <div className="flex space-x-3 pt-4">
        <button
          type="submit"
          disabled={isLoading}
          className="btn-primary flex-1"
        >
          {isLoading ? (
            <>
              <div className="spinner h-4 w-4 mr-2"></div>
              Predicting...
            </>
          ) : (
            <>
              <Play className="h-4 w-4 mr-2" />
              Predict Species
            </>
          )}
        </button>

        <button
          type="button"
          onClick={handleReset}
          className="btn-outline"
          disabled={isLoading}
        >
          <RotateCcw className="h-4 w-4 mr-2" />
          Reset
        </button>
      </div>

      {/* Sample Data Info */}
      <div className="bg-gray-50 rounded-lg p-3 mt-4">
        <h4 className="text-sm font-medium text-gray-700 mb-2">Sample Data Examples:</h4>
        <div className="text-xs text-gray-600 space-y-1">
          <div><strong>Setosa:</strong> SL: 5.1, SW: 3.5, PL: 1.4, PW: 0.2</div>
          <div><strong>Versicolor:</strong> SL: 7.0, SW: 3.2, PL: 4.7, PW: 1.4</div>
          <div><strong>Virginica:</strong> SL: 6.3, SW: 3.3, PL: 6.0, PW: 2.5</div>
        </div>
      </div>
    </form>
  );
};

export default IrisForm; 


Overwriting web/src/components/IrisForm.jsx


In [60]:
%%writefile web/src/components/MLModelFrontend.jsx
import React, { useState, useEffect, useRef } from 'react';
import { Play, RefreshCw, Database, TrendingUp } from 'lucide-react';
import toast from 'react-hot-toast';
import { apiService } from '../services/api';
import IrisForm from './IrisForm';
import CancerForm from './CancerForm';
import ResultsDisplay from './ResultsDisplay';
import ModelTraining from './ModelTraining';

// 🆕 Friendly name helper ------------------------------------------
const prettyModelName = (key) => {
  switch (key) {
    case 'iris_random_forest':   return 'Iris – Random Forest';
    case 'iris_logreg':          return 'Iris – Logistic Regression';
    case 'breast_cancer_bayes':  return 'Breast Cancer – Bayesian';
    case 'breast_cancer_stub':   return 'Breast Cancer – LogReg';
    default:                     return key.replace(/_/g, ' ');
  }
};

// ---- keys we care about from backend ---------------------------------------
const STATUS_KEYS = [
  'iris_random_forest',
  'iris_logreg',
  'breast_cancer_bayes',
  'breast_cancer_stub',
];

/**
 * Sanitize backend model_status payload → {model_name: status_string}.
 * Drops large metadata entries (e.g., *_dep_audit) that crash rendering.
 */
function sanitizeModelStatus(raw) {
  if (!raw || typeof raw !== 'object') {
    console.warn('[ModelStatus] sanitize: non-object payload:', raw);
    return {};
  }

  const filtered = {};
  for (const key of STATUS_KEYS) {
    const val = raw[key];
    if (typeof val === 'string') {
      filtered[key] = val;
    } else if (val !== undefined) {
      console.warn('[ModelStatus] dropping non-string value for', key, val);
    }
  }

  // Log any unexpected keys the backend sent (debug visibility)
  for (const [k, v] of Object.entries(raw)) {
    if (!STATUS_KEYS.includes(k)) {
      console.debug('[ModelStatus] ignoring extra key from backend:', k, v);
    }
  }

  return filtered;
}

const MLModelFrontend = ({ backendReady }) => {
  const [selectedDataset, setSelectedDataset] = useState('iris');
  const [predictions, setPredictions] = useState(null);
  const [isLoading, setIsLoading] = useState(false);
  const [isTraining, setIsTraining] = useState(false);
  const [apiStatus, setApiStatus] = useState('checking');
  const [modelStatus, setModelStatus] = useState({});
  const [selectedModelType, setSelectedModelType] = useState({
    iris: 'rf',
    cancer: 'bayes'
  });

  const intervalRef = useRef(null);

  useEffect(() => {
    // Only start polling once backend indicates readiness (reduces noise)
    if (!backendReady) {
      setApiStatus('loading');
      return;
    }

    checkModelStatus(); // immediate
    if (!intervalRef.current) {
      intervalRef.current = setInterval(checkModelStatus, 4000); // slower cadence
    }

    return () => {
      if (intervalRef.current) {
        clearInterval(intervalRef.current);
        intervalRef.current = null;
      }
    };
  }, [backendReady]);

  const checkModelStatus = async () => {
    try {
      const response = await apiService.getReadyFull();
      console.debug('[checkModelStatus] raw /ready/full response:', response);

      const filtered = sanitizeModelStatus(response.model_status);
      console.debug('[checkModelStatus] filtered model_status:', filtered);

      setModelStatus(filtered);

      if (response.ready) {
        const values = Object.values(filtered);
        const anyFailed = values.includes('failed');
        const allLoaded =
          values.length > 0 && values.every((v) => v === 'loaded');

        if (allLoaded) {
          setApiStatus('healthy');
        } else if (anyFailed) {
          setApiStatus('warning');
        } else {
          setApiStatus('loading');
        }
      } else {
        setApiStatus('error');
      }
    } catch (error) {
      console.error('Failed to check model status:', error);
      setApiStatus('error');
    }
  };

  const handlePredict = async (formData) => {
    setIsLoading(true);
    setPredictions(null);

    try {
      let response;
      if (selectedDataset === 'iris') {
        response = await apiService.predictIris(formData);
      } else {
        response = await apiService.predictCancer(formData);
      }

      setPredictions(response);
      toast.success('Prediction completed successfully!');
    } catch (error) {
      console.error('Prediction error:', error);
      toast.error(`Prediction failed: ${error.message}`);
    } finally {
      setIsLoading(false);
    }
  };

  const handleTrainModel = async () => {
    setIsTraining(true);
    try {
      const modelType = selectedModelType[selectedDataset];
      let response;

      if (selectedDataset === 'iris') {
        response = await apiService.trainIris(modelType);
      } else if (selectedDataset === 'cancer') {
        response = await apiService.trainCancer(modelType);
      }

      toast.success(`Training started for ${selectedDataset} (${modelType})! This may take a few minutes...`);
      console.log('Training response:', response);
    } catch (error) {
      console.error('Training error:', error);
      toast.error(`Training failed: ${error.message}`);
    } finally {
      setIsTraining(false);
    }
  };

  const datasets = [
    {
      id: 'iris',
      name: 'Iris Classification',
      description: 'Classify iris flowers into species based on measurements',
      features: ['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width'],
      classes: ['Setosa', 'Versicolor', 'Virginica']
    },
    {
      id: 'cancer',
      name: 'Breast Cancer Diagnosis',
      description: 'Predict malignant vs benign breast cancer diagnosis',
      features: ['30 diagnostic features'],
      classes: ['Malignant', 'Benign']
    }
  ];

  const currentDataset = datasets.find(d => d.id === selectedDataset);

  const statusColor = {
    healthy: 'bg-green-100 text-green-800',
    warning: 'bg-orange-100 text-orange-800',
    error: 'bg-red-100 text-red-800',
    loading: 'bg-yellow-100 text-yellow-800',
    checking: 'bg-gray-100 text-gray-800'
  }[apiStatus] || 'bg-gray-100 text-gray-800';

  const statusDot = {
    healthy: 'bg-green-500',
    warning: 'bg-orange-500',
    error: 'bg-red-500',
    loading: 'bg-yellow-500',
    checking: 'bg-gray-500'
  }[apiStatus] || 'bg-gray-500';

  const statusText = {
    healthy: 'All Models Ready',
    warning: 'Some Models Failed',
    error: 'API Error',
    loading: 'Models Loading…',
    checking: 'Checking…'
  }[apiStatus] || 'Checking…';

  return (
    <div className="space-y-6">
      {/* Header */}
      <div className="bg-white rounded-lg shadow-sm border border-gray-200 p-6">
        <div className="flex items-center justify-between">
          <div>
            <h2 className="text-2xl font-bold text-gray-900">Machine Learning Models</h2>
            <p className="text-gray-600 mt-1">
              Select a dataset and make predictions using trained models
            </p>
          </div>
          <div className="flex items-center space-x-2">
            <div className={`flex items-center space-x-2 px-3 py-1 rounded-full text-sm ${statusColor}`}>
              <div className={`w-2 h-2 rounded-full ${statusDot}`}></div>
              <span>{statusText}</span>
            </div>
            <button
              onClick={checkModelStatus}
              className="btn-outline btn-sm"
              disabled={apiStatus === 'checking'}
            >
              <RefreshCw className="h-4 w-4 mr-2" />
              Refresh
            </button>
          </div>
        </div>

        {/* Model Status Details */}
        {Object.keys(modelStatus).length > 0 ? (
          <div className="mt-4 grid grid-cols-1 md:grid-cols-2 gap-4">
            {Object.entries(modelStatus).map(([model, status]) => (
              <div
                key={model}
                className="flex items-center justify-between p-2 bg-gray-50 rounded"
              >
                <span className="text-sm font-medium text-gray-700">
                  {prettyModelName(model)}
                </span>
                <span
                  className={`text-xs px-2 py-1 rounded-full ${
                    status === 'loaded'
                      ? 'bg-green-100 text-green-800'
                      : status === 'training'
                      ? 'bg-blue-100 text-blue-800'
                      : status === 'failed'
                      ? 'bg-red-100 text-red-800'
                      : 'bg-gray-100 text-gray-800'
                  }`}
                >
                  {status}
                </span>
              </div>
            ))}
          </div>
        ) : (
          <div className="mt-4 text-sm text-gray-500 italic">
            No model status yet…
          </div>
        )}
      </div>

      {/* Dataset Selection */}
      <div className="grid grid-cols-1 md:grid-cols-2 gap-6">
        {datasets.map((dataset) => (
          <div
            key={dataset.id}
            className={`card cursor-pointer transition-all duration-200 hover:shadow-md ${
              selectedDataset === dataset.id
                ? 'ring-2 ring-primary-500 border-primary-200'
                : 'hover:border-gray-300'
            }`}
            onClick={() => setSelectedDataset(dataset.id)}
          >
            <div className="flex items-start space-x-3">
              <div className={`p-2 rounded-lg ${
                selectedDataset === dataset.id
                  ? 'bg-primary-100 text-primary-600'
                  : 'bg-gray-100 text-gray-600'
              }`}>
                <Database className="h-5 w-5" />
              </div>
              <div className="flex-1">
                <h3 className="font-medium text-gray-900">{dataset.name}</h3>
                <p className="text-sm text-gray-600 mt-1">{dataset.description}</p>
                <div className="mt-2 text-xs text-gray-500">
                  <div>Features: {dataset.features.join(', ')}</div>
                  <div className="mt-1">Classes: {dataset.classes.join(', ')}</div>
                </div>
              </div>
              {selectedDataset === dataset.id && (
                <div className="text-primary-600">
                  <TrendingUp className="h-5 w-5" />
                </div>
              )}
            </div>
          </div>
        ))}
      </div>

      {/* Prediction Form */}
      <div className="grid grid-cols-1 lg:grid-cols-2 gap-6">
        <div className="card">
          <div className="card-header">
            <h3 className="text-lg font-medium text-gray-900">
              {currentDataset?.name} Prediction
            </h3>
            <p className="text-sm text-gray-600">
              Enter values to get a prediction from the trained model
            </p>
          </div>

          {selectedDataset === 'iris' ? (
            <IrisForm 
              onPredict={handlePredict} 
              isLoading={isLoading}
              onModelTypeChange={(modelType) => setSelectedModelType(prev => ({ ...prev, iris: modelType }))}
            />
          ) : (
            <CancerForm 
              onPredict={handlePredict} 
              isLoading={isLoading}
              onModelTypeChange={(modelType) => setSelectedModelType(prev => ({ ...prev, cancer: modelType }))}
            />
          )}
        </div>

        {/* Results */}
        <div className="card">
          <div className="card-header">
            <h3 className="text-lg font-medium text-gray-900">Prediction Results</h3>
            <p className="text-sm text-gray-600">
              Model predictions and confidence scores
            </p>
          </div>

          <ResultsDisplay 
            predictions={predictions} 
            dataset={selectedDataset}
            isLoading={isLoading}
          />
        </div>
      </div>

      {/* Model Training */}
      <ModelTraining 
        dataset={selectedDataset}
        onTrain={handleTrainModel}
        isTraining={isTraining}
        setIsTraining={setIsTraining}
      />
    </div>
  );
};

export default MLModelFrontend; 





Overwriting web/src/components/MLModelFrontend.jsx


In [61]:
%%writefile web/src/App.css
/* App.css - Minimal styles for the ML Dashboard */

.App {
  min-height: 100vh;
}

/* Custom scrollbar for better UX */
::-webkit-scrollbar {
  width: 8px;
}

::-webkit-scrollbar-track {
  background: #f1f1f1;
}

::-webkit-scrollbar-thumb {
  background: #c1c1c1;
  border-radius: 4px;
}

::-webkit-scrollbar-thumb:hover {
  background: #a8a8a8;
}

/* Smooth transitions for interactive elements */
.card {
  transition: all 0.2s ease-in-out;
}

.card:hover {
  transform: translateY(-1px);
}

/* Focus styles for accessibility */
.form-input:focus,
.btn-primary:focus,
.btn-secondary:focus,
.btn-outline:focus {
  outline: 2px solid #3b82f6;
  outline-offset: 2px;
}

/* Loading animation */
@keyframes spin {
  to {
    transform: rotate(360deg);
  }
}

.spinner {
  animation: spin 1s linear infinite;
  /* Fallback size and styling in case Tailwind doesn't process */
  width: 2.5rem;
  height: 2.5rem;
  border: 4px solid #e5e7eb;
  border-top-color: #2563eb;
  border-radius: 50%;
}

/* Responsive text sizing */
@media (max-width: 640px) {
  .text-2xl {
    font-size: 1.5rem;
  }

  .text-lg {
    font-size: 1.125rem;
  }
}


Overwriting web/src/App.css


In [62]:
%%writefile web/src/App.jsx
import React, { useState, useEffect, useRef } from 'react';
import { BrowserRouter as Router, Routes, Route, Navigate } from 'react-router-dom';
import { Toaster } from 'react-hot-toast';
import MLModelFrontend from './components/MLModelFrontend';
import Login from './components/Login';
import Layout from './components/Layout';
import { apiService } from './services/api';
import './App.css';

function App() {
  const [isAuthenticated, setIsAuthenticated] = useState(false);
  const [isLoading, setIsLoading] = useState(true);
  const [user, setUser] = useState(null);
  const [backendReady, setBackendReady] = useState(false);

  // unified poller ref so we can cancel / avoid duplicates
  const pollerRef = useRef(null);
  const destroyedRef = useRef(false);

  // --- DEBUG DIAGNOSTICS (safe to remove later) -----------------
  if (import.meta.env.DEV) {
    // This will run every render – fine for dev introspection
    // eslint-disable-next-line no-console
    console.debug('[App:render]', {
      path: window.location.pathname,
      isAuthenticated,
      isLoading,
      backendReady
    });
  }

  useEffect(() => {
    destroyedRef.current = false;
    (async () => {
      await checkAuthStatus();
      startUnifiedReadinessPoll();
    })();

    return () => {
      destroyedRef.current = true;
      if (pollerRef.current) {
        clearTimeout(pollerRef.current);
        pollerRef.current = null;
      }
    };
    // empty dep array – intentional; StrictMode double-mount safe because cleanup runs
  }, []);

  const startUnifiedReadinessPoll = async (attempt = 0) => {
    if (destroyedRef.current) return;
    try {
      const res = await apiService.getReadyFull(); // use full – includes model_status
      // eslint-disable-next-line no-console
      console.debug('[readiness] attempt', attempt, res);
      if (res?.ready) {
        setBackendReady(true);
        return; // stop polling
      }
    } catch (err) {
      console.error('[readiness] poll error', err);
    }
    const delay = Math.min(1500 * 2 ** attempt, 8000);
    pollerRef.current = setTimeout(() => startUnifiedReadinessPoll(attempt + 1), delay);
  };

  const checkAuthStatus = async () => {
    const token = localStorage.getItem('jwt');
    if (!token) {
      setIsLoading(false);
      return;
    }
    try {
      await apiService.getHealth(); // lightweight probe
      setUser({ username: 'authenticated' });
      setIsAuthenticated(true);
    } catch (err) {
      console.warn('[auth] stored token invalid – clearing', err);
      localStorage.removeItem('jwt');
      setIsAuthenticated(false);
    } finally {
      setIsLoading(false);
    }
  };

  const handleLogin = async (credentials) => {
    try {
      const response = await apiService.login(credentials);
      localStorage.setItem('jwt', response.access_token);
      setUser({ username: credentials.username });
      setIsAuthenticated(true);
      return { success: true };
    } catch (error) {
      return { success: false, error: error.message };
    }
  };

  const handleLogout = () => {
    localStorage.removeItem('jwt');
    setUser(null);
    setIsAuthenticated(false);
  };

  if (isLoading) {
    return (
      <div className="min-h-screen flex items-center justify-center bg-gray-50">
        {/* fallback size so spinner always visible */}
        <div className="spinner" style={{ width: 40, height: 40 }} />
      </div>
    );
  }

  return (
    <Router>
      <div className="App">
        <Toaster
          position="top-right"
          toastOptions={{
            duration: 4000,
            style: { background: '#363636', color: '#fff' }
          }}
        />
        <Routes>
          <Route
            path="/login"
            element={
              isAuthenticated
                ? <Navigate to="/" replace />
                : <Login onLogin={handleLogin} backendReady={backendReady} />
            }
          />
          <Route
            path="/"
            element={
              isAuthenticated
                ? (
                  <Layout user={user} onLogout={handleLogout}>
                    <MLModelFrontend backendReady={backendReady} />
                  </Layout>
                )
                : <Navigate to="/login" replace />
            }
          />
          <Route
            path="/dashboard"
            element={
              isAuthenticated
                ? (
                  <Layout user={user} onLogout={handleLogout}>
                    <MLModelFrontend backendReady={backendReady} />
                  </Layout>
                )
                : <Navigate to="/login" replace />
            }
          />
        </Routes>
      </div>
    </Router>
  );
}

export default App; 







Overwriting web/src/App.jsx


In [63]:
%%writefile web/src/index.css
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700&display=swap');

@tailwind base;
@tailwind components;
@tailwind utilities;

@layer base {
  html { font-family: 'Inter', ui-sans-serif, system-ui; }
  body { @apply bg-gray-50 text-gray-900 antialiased; }
}


@layer components {
  .btn-primary {
    @apply bg-primary-600 hover:bg-primary-700 text-white font-medium py-2 px-4 rounded-lg transition-colors duration-200 focus:outline-none focus:ring-2 focus:ring-primary-500 focus:ring-offset-2;
  }
  
  .btn-secondary {
    @apply bg-secondary-200 hover:bg-secondary-300 text-secondary-800 font-medium py-2 px-4 rounded-lg transition-colors duration-200 focus:outline-none focus:ring-2 focus:ring-secondary-500 focus:ring-offset-2;
  }
  
  .btn-outline {
    @apply border border-gray-300 hover:bg-gray-50 text-gray-700 font-medium py-2 px-4 rounded-lg transition-colors duration-200 focus:outline-none focus:ring-2 focus:ring-primary-500 focus:ring-offset-2;
  }
  
  .btn-sm {
    @apply py-1.5 px-3 text-sm;
  }
  
  .btn-lg {
    @apply py-3 px-6 text-lg;
  }
  
  .card {
    @apply bg-white rounded-lg shadow-sm border border-gray-200 p-6;
  }
  
  .card-header {
    @apply border-b border-gray-200 pb-4 mb-4;
  }
  
  .form-input {
    @apply block w-full px-3 py-2 border border-gray-300 rounded-md shadow-sm placeholder-gray-400 focus:outline-none focus:ring-primary-500 focus:border-primary-500 sm:text-sm;
  }
  
  .form-label {
    @apply block text-sm font-medium text-gray-700 mb-1;
  }
  
  .form-group {
    @apply mb-4;
  }
  
  .alert-success {
    @apply bg-green-50 border border-green-200 text-green-800 px-4 py-3 rounded-lg;
  }
  
  .alert-error {
    @apply bg-red-50 border border-red-200 text-red-800 px-4 py-3 rounded-lg;
  }
  
  .alert-warning {
    @apply bg-yellow-50 border border-yellow-200 text-yellow-800 px-4 py-3 rounded-lg;
  }
  
  .alert-info {
    @apply bg-blue-50 border border-blue-200 text-blue-800 px-4 py-3 rounded-lg;
  }
  
  .table {
    @apply min-w-full divide-y divide-gray-200;
  }
  
  .table-header {
    @apply bg-gray-50;
  }
  
  .table-header th {
    @apply px-6 py-3 text-left text-xs font-medium text-gray-500 uppercase tracking-wider;
  }
  
  .table-body {
    @apply bg-white divide-y divide-gray-200;
  }
  
  .table-body td {
    @apply px-6 py-4 whitespace-nowrap text-sm text-gray-900;
  }
  
  .spinner {
    @apply animate-spin rounded-full h-6 w-6 border-2 border-gray-200 border-t-primary-600;
  }
  
  .badge {
    @apply inline-flex items-center px-2.5 py-0.5 rounded-full text-xs font-medium;
  }
  
  .badge-primary {
    @apply bg-primary-100 text-primary-800;
  }
  
  .badge-secondary {
    @apply bg-secondary-100 text-secondary-800;
  }
  
  .badge-success {
    @apply bg-green-100 text-green-800;
  }
  
  .badge-error {
    @apply bg-red-100 text-red-800;
  }
  
  .badge-warning {
    @apply bg-yellow-100 text-yellow-800;
  }
}


Overwriting web/src/index.css


In [64]:
%%writefile web/tailwind.config.js
/** @type {import('tailwindcss').Config} */
export default {
  content: [
    "./index.html",
    "./src/**/*.{js,ts,jsx,tsx}",
  ],
  theme: {
    extend: {
      colors: {
        primary: {
          50: '#eff6ff',
          100: '#dbeafe',
          200: '#bfdbfe',
          300: '#93c5fd',
          400: '#60a5fa',
          500: '#3b82f6',
          600: '#2563eb',
          700: '#1d4ed8',
          800: '#1e40af',
          900: '#1e3a8a',
        },
        secondary: {
          50: '#f8fafc',
          100: '#f1f5f9',
          200: '#e2e8f0',
          300: '#cbd5e1',
          400: '#94a3b8',
          500: '#64748b',
          600: '#475569',
          700: '#334155',
          800: '#1e293b',
          900: '#0f172a',
        }
      },
      fontFamily: {
        sans: ['Inter', 'ui-sans-serif', 'system-ui'],
      },
      animation: {
        'fade-in': 'fadeIn 0.5s ease-in-out',
        'slide-up': 'slideUp 0.3s ease-out',
      },
      keyframes: {
        fadeIn: {
          '0%': { opacity: '0' },
          '100%': { opacity: '1' },
        },
        slideUp: {
          '0%': { transform: 'translateY(10px)', opacity: '0' },
          '100%': { transform: 'translateY(0)', opacity: '1' },
        },
      },
    },
  },
  plugins: [],
}



Overwriting web/tailwind.config.js


In [65]:
%%writefile web/src/main.tsx
import React from 'react'
import ReactDOM from 'react-dom/client'
import App from './App.jsx'
import './index.css'
import { installDebug } from './debug/GlobalDebug.js'

// Removed runtime-config loader & verification (simplified path).

ReactDOM.createRoot(document.getElementById('root')!).render(
  <React.StrictMode>
    <App />
  </React.StrictMode>
)



Overwriting web/src/main.tsx


%%writefile web/src/debug/GlobalDebug.js
export function installDebug() {
  if (window.__debugInstalled) return;
  window.__debugInstalled = true;

  // Track interval creation
  let intervalCounter = 0;
  const intervalTracker = new Map();

  window.addEventListener('error', (e) => {
    console.error('[GlobalError]', e.error || e.message);
    showOverlay(e.message);
  });

  window.addEventListener('unhandledrejection', (e) => {
    console.error('[GlobalRejection]', e.reason);
    showOverlay('Unhandled Promise: ' + (e.reason?.message || e.reason));
  });

  // Override setInterval to track creation
  const origSetInterval = window.setInterval;
  window.setInterval = function(fn, ms, ...rest) {
    const id = origSetInterval(fn, ms, ...rest);
    intervalCounter++;
    const stack = new Error().stack.split('\n').slice(1, 4).join('\n');
    console.log('[IntervalCreated]', { 
      id, 
      ms, 
      counter: intervalCounter,
      stack 
    });
    intervalTracker.set(id, { fn: fn.toString(), ms, created: Date.now() });
    return id;
  };

  // Override clearInterval to track cleanup
  const origClearInterval = window.clearInterval;
  window.clearInterval = function(id) {
    origClearInterval(id);
    if (intervalTracker.has(id)) {
      console.log('[IntervalCleared]', { 
        id, 
        duration: Date.now() - intervalTracker.get(id).created 
      });
      intervalTracker.delete(id);
    }
  };

  // Add visual debug info
  function showOverlay(msg) {
    let el = document.getElementById('__debug_overlay');
    if (!el) {
      el = document.createElement('div');
      el.id = '__debug_overlay';
      el.style.cssText = 'position:fixed;top:0;left:0;right:0;background:#b91c1c;color:#fff;padding:8px;z-index:99999;font:12px/1.4 monospace;';
      document.body.appendChild(el);
    }
    el.textContent = '[ERROR] ' + msg;
  }

  // Log initial state
  console.log('[Debug] Global debug instrumentation installed');
} 

In [66]:
%%writefile web/src/vite-env.d.ts
/// <reference types="vite/client" />


Overwriting web/src/vite-env.d.ts
